In [17]:
import pandas as pd 

In [18]:
df = pd.read_csv('gemstone.csv')

In [19]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [20]:
x = df.drop('price', axis=1)
y = df['price']

In [21]:
x,y 

(            id  carat        cut color clarity  depth  table     x     y     z
 0            0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55
 1            1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05
 2            2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50
 3            3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71
 4            4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77
 ...        ...    ...        ...   ...     ...    ...    ...   ...   ...   ...
 193568  193568   0.31      Ideal     D    VVS2   61.1   56.0  4.35  4.39  2.67
 193569  193569   0.70    Premium     G    VVS2   60.3   58.0  5.75  5.77  3.47
 193570  193570   0.73  Very Good     F     SI1   63.1   57.0  5.72  5.75  3.62
 193571  193571   0.34  Very Good     D     SI1   62.9   55.0  4.45  4.49  2.81
 193572  193572   0.71       Good     E     SI2   60.8   64.0  5.73  5.71  3.48
 
 [193573 rows x 10 columns],
 0       

## divide catogerical and numerical columns seprate

In [22]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 14.8+ MB


In [23]:
catogerical = []
numreical = []

for i in x.columns:
    if x[i].dtypes =='object':
        catogerical.append(i)
    else:
        numreical.append(i)


In [24]:
catogerical,numreical

(['cut', 'color', 'clarity'], ['id', 'carat', 'depth', 'table', 'x', 'y', 'z'])

In [25]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [26]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [27]:
## create a pipeline for numerical data
num_pipline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scale', StandardScaler()),
    ]
)

## create a pipeline for categorical data
cat_pipline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('rank', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler', StandardScaler())

])

## create a full pipeline(combining both numerical and catogerical pipline both togather)
## columns transformer are use for deviding numerical columns into numerical pipline and catogerical columns into catogerical pipline


full_pipline = ColumnTransformer ([

    ('num', num_pipline, numreical),
    ('cat', cat_pipline, catogerical)
])


## train test split 

In [28]:
from  sklearn.model_selection import train_test_split

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [30]:
## we created this fit transform and tranasform because we want to do feture engineering autometically on our data 

x_train = pd.DataFrame(full_pipline.fit_transform(x_train),columns = full_pipline.get_feature_names_out()).head()
x_test = pd.DataFrame(full_pipline.transform(x_test),columns = full_pipline.get_feature_names_out()).head()